In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("go out or not - person.csv")

In [2]:
data.head()

,Outlook,Temperature,Humidity,Windy,Go Out
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes


In [4]:
data.shape[0]

14

In [5]:
def ginni_impurity(dataset):
    p1 = sum(dataset["Go Out"] == "Yes") / dataset.shape[0]
    p2 = sum(dataset["Go Out"] == "No") / dataset.shape[0]
    return 1 - pow(p1,2) - pow(p2,2)

In [6]:
ginni_impurity(data)

0.4591836734693877

In [7]:
sub = data[data.Temperature == "Hot"]
sub

,Outlook,Temperature,Humidity,Windy,Go Out
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
12,Overcast,Hot,Normal,False,Yes


In [8]:
ginni_impurity(sub)

0.5

In [13]:
def which_feature_to_split_on(dataset):
    information_gain = []
    impurity_without_split = ginni_impurity(dataset)
    for column in dataset.columns[:-1]:
        unique_values = dataset[column].unique()
        impurity_value = 0
        for unique in unique_values:
            current_data = dataset[dataset[column] == unique]
            current_impurity = ginni_impurity(current_data)
            impurity_value += (current_data.shape[0] / dataset.shape[1]) * current_impurity
        information_gain.append(impurity_without_split - impurity_value)
    return dataset.columns[:-1][np.argmax(information_gain)]

In [14]:
def run_decision_tree(main_data):
    feature = which_feature_to_split_on(main_data)
    print("Split the data on...", feature)
    for unique in main_data[feature].unique():
        curr_feature = which_feature_to_split_on(main_data[main_data[feature] == unique])
        if curr_feature != feature:
            print("For ", feature," = ", unique, "split on ", curr_feature)
            run_decision_tree(main_data[main_data[feature] == unique])
        else:
            print("For ", feature, " = ", unique, " predict ", main_data[main_data[feature] == unique]["Go Out"].mode().values[0])

In [15]:
run_decision_tree(data)

Split the data on... Outlook
For  Outlook  =  Rainy split on  Humidity
Split the data on... Humidity
For  Humidity  =  High split on  Outlook
Split the data on... Outlook
For  Outlook  =  Rainy  predict  No
For  Humidity  =  Normal split on  Outlook
Split the data on... Outlook
For  Outlook  =  Rainy  predict  Yes
For  Outlook  =  Overcast  predict  Yes
For  Outlook  =  Sunny split on  Windy
Split the data on... Windy
For  Windy  =  False split on  Outlook
Split the data on... Outlook
For  Outlook  =  Sunny  predict  Yes
For  Windy  =  True split on  Outlook
Split the data on... Outlook
For  Outlook  =  Sunny  predict  No


In [16]:
from sklearn.tree import DecisionTreeClassifier

In [17]:
data

,Outlook,Temperature,Humidity,Windy,Go Out
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes
5,Sunny,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Rainy,Mild,High,False,No
8,Rainy,Cool,Normal,False,Yes
9,Sunny,Mild,Normal,False,Yes


In [19]:
data.columns

Index(['Outlook', 'Temperature', 'Humidity', 'Windy', 'Go Out'], dtype='object')

In [20]:
for column in data.columns:
    data[column] = data[column].map({value:key for key,value in enumerate(data[column].unique())})

In [21]:
data

,Outlook,Temperature,Humidity,Windy,Go Out
0,0,0,0,0,0
1,0,0,0,1,0
2,1,0,0,0,1
3,2,1,0,0,1
4,2,2,1,0,1
5,2,2,1,1,0
6,1,2,1,1,1
7,0,1,0,0,0
8,0,2,1,0,1
9,2,1,1,0,1


In [22]:
X = data[data.columns[:-1]]
y = data[data.columns[-1]]

In [23]:
X

,Outlook,Temperature,Humidity,Windy
0,0,0,0,0
1,0,0,0,1
2,1,0,0,0
3,2,1,0,0
4,2,2,1,0
5,2,2,1,1
6,1,2,1,1
7,0,1,0,0
8,0,2,1,0
9,2,1,1,0


In [24]:
y

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Go Out, dtype: int64

In [25]:
clf = DecisionTreeClassifier()
clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [27]:
clf.predict(X) - y

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
Name: Go Out, dtype: int64